In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [5]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    # MessagePlaceholder : 누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메시지를 가질 수 있다.
    # variable_name 변수명을 주는 것이다.
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}")
])

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory)|prompt|llm

def invoke_chain(question):
    result = chain.invoke({
        "question" : question
    })
    memory.save_context({"input":question},{"output":result.content})
    print(result)


In [6]:
invoke_chain("My name is Nico")

{'question': 'My name is Nico'}
content='Nice to meet you, Nico! How can I assist you today?'


In [7]:
invoke_chain("What is my name?")

{'question': 'What is my name?'}
content='Your name is Nico.'
